### Dependencies

In [1]:
# Import dependencies
import requests
import json
import pandas as pd
import numpy as np
import time

from pprint import pprint
from api_key import api_key

### Extract and Transform CSV Data

In [ ]:
# Path to resources
movies_path = "Resources/movies.csv"
links_path = "Resources/links.csv"

# Read the csvs
movies_df = pd.read_csv(movies_path)
links_df = pd.read_csv(links_path)

# Merge the two csvs into one Pandas Dataframe
merge_df = pd.merge(movies_df, links_df, on = "movieId", how = "outer")

merge_df

In [ ]:
# Drop N/As
cleaned_df = merge_df.dropna()

# Add 'tt' to the IMDb IDs for the API response
cleaned_df.imdbId = cleaned_df.imdbId.astype(str)
cleaned_df.imdbId = 'tt' + cleaned_df.imdbId.str.zfill(7)

# Remove unnecessary columns
del cleaned_df['genres']
del cleaned_df['tmdbId']

# Set index to movieID
cleaned_df = cleaned_df.set_index('movieId')

cleaned_df

In [ ]:
# Store as a CSV to load for API process
cleaned_df.to_csv("Resources/cleaned_movies.csv")

### Extract API Data

In [ ]:
# Read the csv of cleaned movies from our csv source
merged_movies = pd.read_csv('Resources/cleaned_movies.csv')
merged_movies.head()

In [ ]:
# Count of records
merged_movies.imdbId.count()

In [ ]:
# URL and API Key strings for requests
url = "http://www.omdbapi.com/?i="
api_key = "&apikey=" + api_key

In [ ]:
### DO NOT RUN ### API REQUEST TAKES A LONG TIME ###
# Empty lists to hold response info
box_office = []
imdb_id = []
title = []
year = []
runtime = []
genre = []
rated = []
language = []
country = []
metascore = []
imdb_rating = []
type = []

counter = 0

# For loop to append response results for each movie in our csv file
for id in merged_movies['imdbId']:
    try:
        response = requests.get(url + id + api_key).json()
        box_office.append(response['BoxOffice'])
        imdb_id.append(response['imdbID'])
        title.append(response['Title'])
        year.append(response['Year'])
        runtime.append(response['Runtime'])
        genre.append(response['Genre'])
        rated.append(response['Rated'])
        language.append(response['Language'])
        country.append(response['Country'])
        metascore.append(response['Metascore'])
        imdb_rating.append(response['imdbRating'])
        type.append(response['Type'])
        counter += 1
        print(f'Processed record: {id}')
    except KeyError:
        print(f'Record {id} missing key information. Skipping...')
    
print(f'The total number of records found was: {counter} out of {merged_movies.imdbId.count()}')

In [ ]:
len(imdb_id), len(title), len(year), len(runtime), len(genre), len(rated), len(language),len(country), len(metascore), len(imdb_rating), len(box_office), len(type)

In [ ]:
#Put the API response into a DataFrame
api_data = pd.DataFrame({
    'IMDbID': imdb_id,
    'Title': title,
    'Year': year,
    'Runtime': runtime,
    'Genre': genre,
    'Rated': rated,
    'Language': language,
    'Country': country,
    'Metascore': metascore,
    'IMDb_Rating': imdb_rating,
    'Box_Office': box_office,
    'Type': type
})

In [ ]:
# Send the API response data to a csv file for cleaning using pandas
api_data.to_csv('Resources/api_data.csv')

### Transform the API Data

In [2]:
# Read the API dataset response for cleaning
df_path = "Resources/api_data.csv"
df = pd.read_csv(df_path)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57210 entries, 0 to 57209
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   57210 non-null  int64  
 1   IMDbID       57210 non-null  object 
 2   Title        57210 non-null  object 
 3   Year         57210 non-null  int64  
 4   Runtime      56960 non-null  object 
 5   Genre        57114 non-null  object 
 6   Rated        40810 non-null  object 
 7   Language     56762 non-null  object 
 8   Country      57100 non-null  object 
 9   Metascore    14130 non-null  float64
 10  IMDb_Rating  57058 non-null  float64
 11  Box_Office   15188 non-null  object 
 12  Type         57210 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usage: 5.7+ MB


In [ ]:
# Remove unnecessary columns and N/A data
del df["Unnamed: 0"]
reduced = df.dropna()

# Fix columns for cleaner data
reduced['Runtime'] = reduced['Runtime'].str.extract('(\d+)').astype(int)
reduced['Box_Office'] = reduced['Box_Office'].str.replace("$", "")
reduced['Box_Office'] = reduced['Box_Office'].str.replace(",", "")
reduced['Box_Office'] = reduced['Box_Office'].astype(int)
reduced.set_index('IMDbID', inplace=True)

reduced

In [ ]:
# Save as a CSV for Postgres Import
reduced.to_csv("Resources/cleaned_api_data.csv")

### Go to movies.sql file for Load process